# Proyecto End to End de Machine Learning 
### Viviendas en venta en Madrid


![imagen idealista fachada](../img/imagenEDA_idealista-fachada.jpg)

Se recaban datos de la web idealista, en febrero de 2025. Se trata de pisos en venta, de la ciudad de Madrid, sus 21 distritos. 

![imagen total viviendas idealista Madrid](../img/idealista_num_viv(02.03.2025).jpg)

<img src="../img/idealista_num_viv(02.03.2025).jpg" alt="imagen total viviendas idealista Madrid" width="600">

<img src="../img/idealista_num_viv(03.03.2025).jpg" alt="imagen total viviendas idealista Madrid" width="600">

El número de viviendas cambia diariamente, como era de suponer.
Finalmente, voy a trabajar con las viviendas del distrito Centro, que en fecha de hoy (9-marzo-2025) son 1792 en mi solicitud a traves de la API, aunque en el portal aparezcan 1976. 

<img src="../img/idealista_num_viv_centro(10.03.2025).jpg" alt="imagen total viviendas idealista Madrid" width="600">

Este primer jupiter notebook contiene todo lo relativo a la carga de los datos, mediante web scraping de la web idealista.

Llegará hasta el punto 3, donde se realiza la división en train y test, o eso creo

## 0. Librerías
 

In [30]:
# importación agrupada de librerías necesarias en este notebook
import pandas as pd
import numpy as np

import sys
import os
from datetime import date

from scipy import stats
from PIL import Image
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns

import requests

#warnings.filterwarnings('ignore')

# Añado el directorio padre (del que está este notebook) a sys.path
sys.path.append(os.path.abspath('../'))
from scripts.utils_agv import ini_inspec, crear_tabla_resumen, categoricas, numericas

## 1. Carga datos

### Datos anteriores, obtenidos desde Kaggle

In [16]:
previo_ejemplo = pd.read_csv('../data/raw/idealista_Madrid_Kaggle.csv')
previo_ejemplo.head(1)

,url,listingUrl,title,id,price,baths,rooms,sqft,description,address,typology,advertiserProfessionalName,advertiserName
0,https://www.idealista.com/inmueble/104027174/,https://www.idealista.com/venta-viviendas/madr...,Piso en venta en calle de Villanueva,104027174,1920000,3,3,183,Residencia única con acabados de la más alta c...,"Recoletos, Madrid",Pisos,Promora Madrid,Promora Madrid


### Obtención de datos "nuevos": Web Scraping Idealista

In [17]:
viv_Mad = pd.read_csv('../data/raw/idealista_Madrid-Ciudad(Centro)_2025-03-10.csv')
viv_Mad.head(1)

,propertyCode,thumbnail,externalReference,numPhotos,floor,price,priceInfo,propertyType,operation,size,...,has3DTour,has360,hasStaging,highlight,savedAd,notes,topNewDevelopment,topPlus,parkingSpace,newDevelopmentFinished
0,107526421,https://img4.idealista.com/blur/WEB_LISTING/0/...,GH202391,43,3,1095000.0,"{'price': {'amount': 1095000.0, 'currencySuffi...",flat,sale,146.0,...,True,False,False,{'groupDescription': 'Top+'},{},[],False,True,NaN,NaN


In [18]:
#comprobación de que no han llegado filas duplicadas
viv_Mad.iloc[[0,50,100,150],:]

,propertyCode,thumbnail,externalReference,numPhotos,floor,price,priceInfo,propertyType,operation,size,...,has3DTour,has360,hasStaging,highlight,savedAd,notes,topNewDevelopment,topPlus,parkingSpace,newDevelopmentFinished
0,107526421,https://img4.idealista.com/blur/WEB_LISTING/0/...,GH202391,43,3,1095000.0,"{'price': {'amount': 1095000.0, 'currencySuffi...",flat,sale,146.0,...,True,False,False,{'groupDescription': 'Top+'},{},[],False,True,NaN,NaN
50,107063820,https://img4.idealista.com/blur/WEB_LISTING/0/...,JGA205341,18,bj,269000.0,"{'price': {'amount': 269000.0, 'currencySuffix...",flat,sale,39.0,...,True,False,False,{'groupDescription': 'Top'},{},[],False,False,NaN,NaN
100,101932319,https://img4.idealista.com/blur/WEB_LISTING/0/...,10007843,52,1,622000.0,"{'price': {'amount': 622000.0, 'currencySuffix...",flat,sale,159.0,...,False,True,False,{'groupDescription': 'Top'},{},[],False,False,NaN,NaN
150,107020358,https://img4.idealista.com/blur/WEB_LISTING/0/...,85571074,47,2,2500000.0,"{'price': {'amount': 2500000.0, 'currencySuffi...",flat,sale,262.0,...,False,False,False,{'groupDescription': 'Top'},{},[],False,False,NaN,NaN


In [19]:
# realizo siempre una copia para recuperar el df por si lo rompo
df = viv_Mad.copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1792 entries, 0 to 1791
Data columns (total 43 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   propertyCode            1792 non-null   int64  
 1   thumbnail               1777 non-null   object 
 2   externalReference       1351 non-null   object 
 3   numPhotos               1792 non-null   int64  
 4   floor                   1727 non-null   object 
 5   price                   1792 non-null   float64
 6   priceInfo               1792 non-null   object 
 7   propertyType            1792 non-null   object 
 8   operation               1792 non-null   object 
 9   size                    1792 non-null   float64
 10  exterior                1768 non-null   object 
 11  rooms                   1792 non-null   int64  
 12  bathrooms               1792 non-null   int64  
 13  address                 1792 non-null   object 
 14  province                1792 non-null   

Comprobaré primero que no me han llegado datos duplicados, y en caso de que los hubiera, los borraré antes de arrastrarlos

In [20]:
df.duplicated().sum()

np.int64(45)

In [21]:
# Filtrar filas duplicadas (manteniendo la primera aparición también para ver el grupo completo)
df_duplicados = df[df.duplicated(keep=False)]

# Ordenar por 'propertyCode' para agrupar visualmente las duplicadas
df_duplicados = df_duplicados.sort_values(by='propertyCode')

df_duplicados


,propertyCode,thumbnail,externalReference,numPhotos,floor,price,priceInfo,propertyType,operation,size,...,has3DTour,has360,hasStaging,highlight,savedAd,notes,topNewDevelopment,topPlus,parkingSpace,newDevelopmentFinished
1298,102412308,https://img4.idealista.com/blur/WEB_LISTING/0/...,82996735,31,4,789000.0,"{'price': {'amount': 789000.0, 'currencySuffix...",flat,sale,101.0,...,False,False,False,NaN,{},[],False,False,NaN,NaN
1302,102412308,https://img4.idealista.com/blur/WEB_LISTING/0/...,82996735,31,4,789000.0,"{'price': {'amount': 789000.0, 'currencySuffix...",flat,sale,101.0,...,False,False,False,NaN,{},[],False,False,NaN,NaN
1238,102476882,https://img4.idealista.com/blur/WEB_LISTING/0/...,NaN,36,3,1785000.0,"{'price': {'amount': 1785000.0, 'currencySuffi...",flat,sale,204.0,...,False,False,False,NaN,{},[],False,False,"{'hasParkingSpace': True, 'isParkingSpaceInclu...",NaN
1251,102476882,https://img4.idealista.com/blur/WEB_LISTING/0/...,NaN,36,3,1785000.0,"{'price': {'amount': 1785000.0, 'currencySuffi...",flat,sale,204.0,...,False,False,False,NaN,{},[],False,False,"{'hasParkingSpace': True, 'isParkingSpaceInclu...",NaN
1294,102519584,https://img4.idealista.com/blur/WEB_LISTING/0/...,83256354,29,2,859000.0,"{'price': {'amount': 859000.0, 'currencySuffix...",flat,sale,88.0,...,False,False,False,NaN,{},[],False,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
851,107519888,https://img4.idealista.com/blur/WEB_LISTING/0/...,W-02Y2IU,29,7,1070000.0,"{'price': {'amount': 1070000.0, 'currencySuffi...",flat,sale,127.0,...,False,True,False,NaN,{},[],False,False,NaN,NaN
1005,107543117,https://img4.idealista.com/blur/WEB_LISTING/0/...,3770e25f-bd43-49f3-8dfe-3fdc58a604ba-0,9,2,495000.0,"{'price': {'amount': 495000.0, 'currencySuffix...",flat,sale,90.0,...,False,False,False,NaN,{},[],False,False,NaN,NaN
999,107543117,https://img4.idealista.com/blur/WEB_LISTING/0/...,3770e25f-bd43-49f3-8dfe-3fdc58a604ba-0,9,2,495000.0,"{'price': {'amount': 495000.0, 'currencySuffix...",flat,sale,90.0,...,False,False,False,NaN,{},[],False,False,NaN,NaN
997,107564811,https://img4.idealista.com/blur/WEB_LISTING/0/...,SAN VICENTE FERRER,21,3,650000.0,"{'price': {'amount': 650000.0, 'currencySuffix...",flat,sale,102.0,...,False,False,False,NaN,{},[],False,False,NaN,NaN


In [22]:
#eliminar duplicados sobreescribiendo
df.drop_duplicates(keep='first', inplace=True)


In [23]:
# vamos a eliminar el índice y usar como índice esta columna
df = df.reset_index(drop=True).set_index("propertyCode")

<img src="../img/idealista_Centro_0-EU-ES-28-07-001-079-01.jpg" alt="imagen zona datos idealista Madrid" width="600">

In [29]:
# codigo prueba colocar la localización de una vivienda
# import matplotlib.pyplot as plt
# import requests
# from PIL import Image

# # Coordenadas de la imagen (límites)
# lat_min, lon_min = 40.403888, -3.725449  # Esquina inferior izquierda
# lat_max, lon_max = 40.431398, -3.689122  # Esquina superior derecha

# # Tamaño de la imagen
# img_width, img_height = 448, 448

# # Coordenada a ubicar en la imagen (Ejemplo: Calle de las Fuentes)
# lat_point, lon_point = 40.417601, -3.70757   # localización head1

# # Cálculo de la posición en píxeles
# x_pixel = ((lon_point - lon_min) / (lon_max - lon_min)) * img_width
# y_pixel = (1 - (lat_point - lat_min) / (lat_max - lat_min)) * img_height  # Invertimos para ajustar

# # Cargar la imagen del mapa
# imagen_url = "https://st3.idealista.com/static/es/img/maps/0-EU-ES-28-07-001-079-01.jpg"
# imagen = Image.open(requests.get(imagen_url, stream=True).raw)

# # Mostrar imagen y marcar punto
# fig, ax = plt.subplots(figsize=(14, 14))
# ax.imshow(imagen)
# ax.scatter(x_pixel, y_pixel, color='red', s=50, marker='o')  # Dibujar punto en rojo
# ax.set_title("Ubicación en la imagen")
# ax.axis("off")  # Ocultar ejes
# plt.show()



In [ ]:
# Ubicar puntos en cada localización de vivienda

# Coordenadas (aproximadas) de la imagen (límites)
lat_min, lon_min = 40.403888, -3.725449  # Esquina inferior izquierda
lat_max, lon_max = 40.431398, -3.689122  # Esquina superior derecha

# Tamaño de la imagen
img_width, img_height = 448, 448

# Cargar la imagen del mapa
imagen_url = "https://st3.idealista.com/static/es/img/maps/0-EU-ES-28-07-001-079-01.jpg"
imagen = Image.open(requests.get(imagen_url, stream=True).raw)

# Mostrar imagen
fig, ax = plt.subplots(figsize=(14, 14))
ax.imshow(imagen)

# Iterar sobre cada fila del DataFrame y dibujar el punto correspondiente
for index, row in df.iterrows():
    lat_point = row['latitude']
    lon_point = row['longitude']

    # Cálculo de la posición en píxeles
    x_pixel = ((lon_point - lon_min) / (lon_max - lon_min)) * img_width
    y_pixel = (1 - (lat_point - lat_min) / (lat_max - lat_min)) * img_height  # Invertimos para ajustar

    # Dibujar punto en rojo
    ax.scatter(x_pixel, y_pixel, color='red', s=50, marker='o')  # Ajusta el tamaño y estilo si es necesario

ax.set_title("Localización aproximada de las viviendas obtenidas")
ax.axis("off")  # Ocultar ejes
plt.show()


In [ ]:
df.T

In [ ]:
df ['parkingSpace'].unique()

In [ ]:
ini_inspec(df)

In [ ]:
df.columns.to_list

In [ ]:
pd.set_option('display.max_colwidth', None)  # Evita que Pandas corte el texto
df.head(5).T

In [ ]:
df['parkingSpace'].unique()

La columna parkingSpace merece un comentario aparte. Dado que mayormente no tienen garaje, y este, cuando existe, no está incluido en el precio, se decide eliminar esa columna.

### Limpieza previa datos obtenidos

Dado que conozco aproximadamente los datos por haber trabajado con un dataset similar en fases anteriores, voy a eliminar columnas que me proporciona el portal Idealista, que son irrelevantes para el trabajo que se plantea.

In [ ]:
col_eliminar = ['thumbnail','externalReference', 'priceInfo', 'operation', 'province', 'municipality',
       'country', 'showAddress', 'url', 'newDevelopment', 'change', 'highlight', 'savedAd', 
       'notes','hasStaging', 'topNewDevelopment', 'parkingSpace' ,'newDevelopmentFinished' ]
df2 = df.drop(col_eliminar, axis=1 )

In [ ]:
pd.set_option('display.max_colwidth', None)  # Evita que Pandas corte el texto
df2.head(5).T

In [ ]:
ini_inspec(df2)

In [ ]:
# Con los datos, función para convertir en csv y guardarlo.
today =  date.today ()
file_path = f'../data/processed/ide_viv_limpieza0_{today}.csv' 

def df_to_csv(df):
    if os.path.exists(file_path):
        print(f"⚠️ El archivo '{file_path}' ya existe. No se sobrescribirá.")
    else:
        df.to_csv(file_path)   #lo guarda en un csv con indice en propertyCode
        print(f"✅ Archivo guardado correctamente como '{file_path}'.")

In [ ]:
#Guarda los datos pre separación Train-test en un csv con nombre establecido.
df_to_csv(df2)

## 2. Definición del problema de Machine Learning

Mi problema de ML consiste en 'predecir' el precio de una vivienda dadas sus características, y las características o campos que figuran en el anuncio que pueda obtener de idealista. A aplicar a viviendas nuevas que aparezcan, en el futuro (problema asociado a la evolución de los precios en el tiempo). Se trata entonces de un modelo de regresión, sin series temporales.

## 3. Target

Dentro de este primer analisis exploratorio de los datos, revisamos la target, para ver en nuestro problema de regresión, si esta tiene una distribución normal o asimétrica.

In [ ]:
df_target = df["price"]
sns.kdeplot(df_target);

La target, el precio, no tiene una distribución normal, sino asimétrica positiva, hacia la derecha. Vamos a probar a realizar algunas transformaciones matemáticas, para ver si puedo normalizarla. 

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(20, 5), sharey=True)

# Original target
sns.histplot(df_target, kde=False, ax=axes[0])
axes[0].set_title("Original target")

# Logaritmic
sns.histplot(np.log(df_target),kde=False, ax=axes[1])
axes[1].set_title("Log")

# Box-cox
sns.histplot(stats.boxcox(df_target)[0],kde=False, ax=axes[2])
axes[2].set_title("Box-Cox");

# Power 2
sns.histplot(np.power(df_target, 2),kde=False, ax=axes[3])
axes[3].set_title("Power 2")

In [ ]:
# Prueba de Shapiro-Wilk para cada transformación
shapiro_original = stats.shapiro(df_target)
shapiro_log = stats.shapiro(np.log(df_target))
shapiro_boxcox = stats.shapiro(stats.boxcox(df_target)[0])
shapiro_power2 = stats.shapiro(np.power(df_target, 2))

# Imprimir resultados
print(f"Shapiro-Wilk Test - Original: p-value = {shapiro_original.pvalue:.10f}")
print(f"Shapiro-Wilk Test - Log: p-value = {shapiro_log.pvalue:.10f}")
print(f"Shapiro-Wilk Test - Box-Cox: p-value = {shapiro_boxcox.pvalue:.10f}")
print(f"Shapiro-Wilk Test - Power 2: p-value = {shapiro_power2.pvalue:.10f}")


Todas las transformaciones tienen p-valores bajos (<0.05), significa que ninguna logra una normalidad perfecta, buscaremos el que tenga mayor p-valor.

In [ ]:
# Grafica Q-Q plot de cada una de las transformaciones
fig, axes = plt.subplots(1, 4, figsize=(20, 5))
#  Original
stats.probplot(df_target, dist="norm", plot=axes[0])
axes[0].set_title("QQ-Plot: Original")

# Log
stats.probplot(np.log(df_target), dist="norm", plot=axes[1])
axes[1].set_title("QQ-Plot: Log")

# Box-Cox
stats.probplot(stats.boxcox(df_target)[0], dist="norm", plot=axes[2])
axes[2].set_title("QQ-Plot: Box-Cox")

# Power 2
stats.probplot(np.power(df_target, 2), dist="norm", plot=axes[3])
axes[3].set_title("QQ-Plot: Power 2")

plt.show()


Ninguna transformación logra una distribución normal, pero el logaritmo y box-cox logran resultados casi idénticos. Ante ambas opciones, sigo la elección más fácil de explicar, la transformación logarítmica. 

___________________________________________________________
Con estos pasos, paso al siguiente notebook donde realizaré y desarrollaré la fase de EDA, desde compresión de variables y analisis de los datos hasta reducción de dimensionalidad

#
[Ir al siguiente notebook: Análisis Exploratorio (EDA)](./02_Analisis_exploratorio.ipynb)
